In [ ]:
###··· RING MERGED DATASET ···### 
import pandas as pd
from matplotlib import pyplot as plt
from seaborn import heatmap
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *

# Part 1: Selecting and analyzing the dataset 
# #Load the dataset
ring_separable = pd.read_csv("A2-ring\A2-ring-separable.txt", sep='\t', header=None)
ring_separable.columns = ["X", "Y", "Result"]
# Select the columns
x_train_unscaled = ring_separable[["X", "Y"]]
y_train = ring_separable[["Result"]]

ring_test = pd.read_csv("A2-ring\A2-ring-test.txt", sep='\t', header=None)
ring_test.columns = ["X", "Y", "Result"]
# Select the columns
x_test_unscaled = ring_test[["X", "Y"]]
y_test = ring_test[["Result"]]

# Scale the dat
scaler = MinMaxScaler()
scaler.fit(x_train_unscaled)
x_train = scaler.transform(x_train_unscaled)
x_test = scaler.transform(x_test_unscaled)

plt.scatter(x_train_unscaled["X"], x_train_unscaled["Y"], c=y_train["Result"].map({0: 'blue', 1: 'orange'}), marker="o")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Scatter Plot ring separated")
plt.show()

In [ ]:
###---Part 2.a: Implement a tensorflow BP model---#
import tensorflow as tf
from tensorflow import keras as keras
from scikeras.wrappers import KerasClassifier
tf.get_logger().setLevel('ERROR')

# Create the model
model = keras.models.Sequential([
    keras.layers.Dense(10, input_shape=(x_train.shape[1],), activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cv_model = KerasClassifier(model=model, verbose=0)
# Perform cross-validation
grid = GridSearchCV(estimator=cv_model,
                    param_grid= {'epochs': [30],'optimizer__learning_rate': [0.001, 0.01, 0.1], 'optimizer__emu_momentum': [0.1, 0.01, 0.001]},
                    cv=5, 
                    scoring='accuracy')

In [ ]:
###···Part 2.b: Implement MLR ···###
import sklearn.linear_model as linear_model
# Create the model
model = linear_model.LogisticRegressionCV(max_iter=10000)
# Perform cross-validation
grid = GridSearchCV(estimator=model,
                    param_grid= {'Cs': [1, 10], 'solver': ['newton-cg', 'lbfgs','saga']},
                    cv=5, 
                    scoring='accuracy')

In [ ]:
###···Part 2.c: Implement SVM ···###
import sklearn.svm as svm
# Create the model
model = svm.SVC()
grid = GridSearchCV(estimator=model,
                    param_grid= {'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01], 'kernel': ['rbf', 'linear']},
                    cv=5, 
                    scoring='accuracy')

In [ ]:
#---Part 3: Evaluate the model and summarize results---#
# Fit the model
grid.fit(x_train, y_train.values.ravel())
# summarize results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Classification error: ", 1 - grid.best_score_)
# Predict the response for test dataset
y_train_pred = grid.predict(x_train)
y_test_pred = grid.predict(x_test)

print("---Train---")
print("Accuracy Score: ", accuracy_score(y_train, y_train_pred))
print("---Test---")
print("Accuracy Score: ", accuracy_score(y_test, y_test_pred))

# Generate the confusion matrix

confusion = confusion_matrix(y_test, y_test_pred, labels=[0, 1])
print(confusion)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
heatmap(confusion, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix test')
plt.show()